In [1]:
import pandas as pd
import numpy as np
import os
# import Levenshtein
import difflib

In [2]:
cell2ind = pd.read_csv('../data/cell2ind.txt', header=None, sep='\t')
gene2ind = pd.read_csv('../data/gene2ind.txt', header=None, sep='\t')
drug2ind = pd.read_csv('../data/drug2ind.txt', header=None, sep='\t')
cell2mutation = pd.read_csv('../data/cell2mutation.txt', header=None, sep=',')
drug2fingerprint = pd.read_csv('../data/drug2fingerprint.txt', header=None, sep=',')
drugcell_ont = pd.read_csv('../data/drugcell_ont.txt', header=None, sep='\t')

train = pd.read_csv('../data/drugcell_train.txt', header=None, sep='\t')
test = pd.read_csv('../data/drugcell_test.txt', header=None, sep='\t')
cellminer = pd.read_csv('../data/out_.txt', header=None, sep='\t')
# cellminer[0] = cellminer[0].str.upper()
cellminer[1] = cellminer[1].str.upper()

# Shape

In [3]:
print('Cell Index:' + str(cell2ind.shape))
print('Gene Index:' + str(gene2ind.shape))
print('Drug Index:' + str(drug2ind.shape))
print('Cell Mutation:' + str(cell2mutation.shape))
print('Drug Fingerprint:' + str(drug2fingerprint.shape))

Cell Index:(1225, 2)
Gene Index:(3008, 2)
Drug Index:(684, 2)
Cell Mutation:(1225, 3008)
Drug Fingerprint:(684, 2048)


In [4]:
len(set(train[0]) & set(test[0]))

622

In [5]:
set(test[0]) - set(train[0])

{'JHU028_LUNG', 'KYSE220_OESOPHAGUS', 'MCC13_SKIN', 'NCCSTCK140_STOMACH'}

In [6]:
len(set(test[0]))

626

In [7]:
len(set(train[1]) & set(test[1]))

479

In [8]:
set(test[1]) - set(train[1])

{'CC(C)(C)C1=CC2=C(C=C1)C=C(C(=C2)C#N)C#N.CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)NC4=CC=C(C=C4)N5CCN(CC5)C)O'}

In [9]:
len(set(test[1]))

480

# The 1st column is identification of cells (genotypes) and the 2nd column is identification of drugs.
# Don't know how to get Cell Line and SMILES pair from cellminer

# Sample data

In [10]:
test.head()
# CELL LINE : SMILES

,0,1,2
0,LS123_LARGE_INTESTINE,CNC(=O)C1=NC=CC(=C1)OC2=CC(=C(C=C2)NC(=O)NC3=C...,0.894224
1,RT4_URINARY_TRACT,C[C@@H](C(=O)N[C@H]1C2=CC=CC=C2CCN(C1=O)C)NC(=...,0.874261
2,SCABER_URINARY_TRACT,C1CCN(C1)C(=O)NC2=CC=CC(=C2)NC3=NC=C(C(=N3)NCC...,0.960072
3,TE617T_SOFT_TISSUE,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCNC(=O)C4=CC5=...,0.987354
4,MOGGCCM_CENTRAL_NERVOUS_SYSTEM,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,0.871997


In [11]:
train[train[0] == 'KYSE450_OESOPHAGUS']

,0,1,2
0,KYSE450_OESOPHAGUS,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,1.006587
67,KYSE450_OESOPHAGUS,COC1=C(C=CC(=C1)C=CC(=O)CC(=O)C=CC2=CC(=C(C=C2...,0.890410
382,KYSE450_OESOPHAGUS,CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=...,1.001322
768,KYSE450_OESOPHAGUS,CCOC(=O)C1=CC=C(C=C1)N2C(=O)C(=CC3=CC=C(O3)[N+...,0.976716
785,KYSE450_OESOPHAGUS,CC1=CC2=C(C=C1)N=C(C3=NC=C(N23)C)NCCN,0.772537
2173,KYSE450_OESOPHAGUS,C1=C(C(=O)NC(=O)N1)F,0.481351
2266,KYSE450_OESOPHAGUS,COC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=C(C=C3)NC(=O)...,0.982268
2885,KYSE450_OESOPHAGUS,CN1CCN(CC1)CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl,1.013487
3538,KYSE450_OESOPHAGUS,CC(C)OC(=O)C1=CN(CC(C2=C1NC3=CC=CC=C32)(C)C)C(...,0.976186
3572,KYSE450_OESOPHAGUS,COC(=O)CNC(=O)C(=O)OC,0.734794


In [12]:
test[test[0] == 'KYSE450_OESOPHAGUS']

,0,1,2
372,KYSE450_OESOPHAGUS,COC1=CC(=C(C=C1)OC)C2=NN=C3N2N=C(CS3)C4=CC(=C(...,0.782034


# Shape

In [13]:
print('Cell Index:' + str(cell2ind.shape))
print('Gene Index:' + str(gene2ind.shape))
print('Drug Index:' + str(drug2ind.shape))
print('Cell Mutation:' + str(cell2mutation.shape))
print('Drug Fingerprint:' + str(drug2fingerprint.shape))
print('Gene Ontology:' + str(drugcell_ont.shape))

Cell Index:(1225, 2)
Gene Index:(3008, 2)
Drug Index:(684, 2)
Cell Mutation:(1225, 3008)
Drug Fingerprint:(684, 2048)
Gene Ontology:(62920, 3)


In [14]:
cell2ind.head()
# index : Cell Line

,0,1
0,0,NCIH810_LUNG
1,1,HN_UPPER_AERODIGESTIVE_TRACT
2,2,COV434_OVARY
3,3,CCRFCEM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
4,4,PECAPJ49_UPPER_AERODIGESTIVE_TRACT


In [15]:
gene2ind.head()
# index : Gene Name
# related to cell2mutation

,0,1
0,0,AADAC
1,1,AADAT
2,2,AAK1
3,3,AARS
4,4,AASDHPPT


In [16]:
drug2ind.head()
# index : SMILES

,0,1
0,0,C[C@H]1CN(C(=O)C2=C(C(=CC=C2)NS(=O)(=O)C3=CC=C...
1,1,C1C2CC3CC1CC(C2)(C3)C4=C(C=C5C=C(C=CC5=C4)C6=C...
2,2,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N/C(=C\3/C4=...
3,3,CC(CS(=O)(=O)C1=CC=C(C=C1)F)(C(=O)NC2=CC(=C(C=...
4,4,C[C@H]1CN(C(=O)C2=C(C=CC(=C2)NS(=O)(=O)C3=CN(C...


In [17]:
cell2mutation.head()
# Mutation for each Cell Line

,0,1,2,3,4,5,6,7,8,9,...,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
drug2fingerprint.head()
# Fingerprint for each drug

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
drugcell_ont.head()

,0,1,2
0,GO:0007005,GO:0007006,default
1,GO:0007005,GO:0008637,default
2,GO:0006281,GO:0006284,default
3,GO:0006281,GO:0006283,default
4,GO:0006281,GO:0019985,default


In [20]:
drugcell_ont.tail()

,0,1,2
62915,GO:2001240,BCL2L1,gene
62916,GO:2001240,AKT1,gene
62917,GO:0008150,LYPLAL1,gene
62918,GO:0008150,ABHD11,gene
62919,GO:0008150,EML4,gene


In [21]:
drugcell_ont[drugcell_ont[1] == 'CFL1']

,0,1,2
5708,GO:0000281,CFL1,gene
5714,GO:0030010,CFL1,gene
7970,GO:0001101,CFL1,gene
8883,GO:0048646,CFL1,gene
10534,GO:0043009,CFL1,gene
10793,GO:0001755,CFL1,gene
12262,GO:0016331,CFL1,gene
12346,GO:0060429,CFL1,gene
12389,GO:0001841,CFL1,gene
14949,GO:0006468,CFL1,gene


In [22]:
cellminer

,0,1
0,METHOTREXATE,CN(CC1CNC2NC(N)NC(N)C2N1)C3CCC(CC3)C(=O)N[C@@H...
1,6-THIOGUANINE,NC1=NC(=S)C2[NH]CNC2N1
2,6-MERCAPTOPURINE,S=C1N=CNC2NC[NH]C12
3,Allopurinol,O=C1NC=NC2[NH]NCC12
4,Chlorambucil,OC(=O)CCCC1CCC(CC1)N(CCCL)CCCL
...,...,...
193,gilteritinib,CCC1NC(C(=O)N)C(NC2CCC(N3CCC(CC3)N4CCN(C)CC4)C...
194,Acalabrutinib,CC#CC(=O)N1CCC[C@H]1C2NC(C3CCC(CC3)C(=O)NC4CCC...
195,Sulfatinib,CN(C)CCNS(=O)(=O)CC1CCCC(NC2NCCC(OC3CCC4[NH]C(...
196,umbralisib,CC(C)OC1CCC(CC1F)C2NN([C@@H](C)C3=C(C(=O)C4CC(...


In [23]:
len(set(train[1]))

680

In [24]:
len(set(train[1]) - set(cellminer[1]))

680

In [25]:
len(set(train[1]) - set(test[1]))

201

In [26]:
train.head()
# CELL LINE : SMILES
# The 1st column is identification of cells (genotypes) and the 2nd column is identification of drugs.
# Don't know how to get Cell Line and smiles pair from cellminer

,0,1,2
0,KYSE450_OESOPHAGUS,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,1.006587
1,SUDHL1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,C1CN(CCN1)C(=O)C2=CC=C(C=C2)/C=C/C3=NNC4=CC=CC...,0.705210
2,OCIAML3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,0.885490
3,SCC9_UPPER_AERODIGESTIVE_TRACT,C1=CC=C2C(=C1)C3=CC=CC=C3N2CCC4=NC5=CC=CC=C5N4,0.955772
4,SNU489_CENTRAL_NERVOUS_SYSTEM,CCCCCCCCCCC(C)(C)C(=O)NC1=C(C=C(C=C1OC)OC)OC,1.012310


In [27]:
for i, j in zip(train[0], train[1]):
    if 'MDA' in i:
        print(i, j)
# Naming rule between cellminer and this ml mthods are completely different...

MDAMB231_BREAST C1=CC=C2C(=C1)C(=CC=N2)CNC3=C(SC=C3)C(=O)NC4=CC=C(C=C4)OC(F)(F)F
MDAMB436_BREAST N.N.[Cl-].[Cl-].[Pt+2]
MDAMB361_BREAST CC1=CC(=NN1)NC2=NC(=NC=C2Cl)N[C@@H](C)C3=NC=C(C=N3)F
MDAMB361_BREAST CC(C)(C)C1=CC(=CC(=C1)C(=O)/C=C/C2=CC=C(C=C2)C(=O)O)C(C)(C)C
MDAMB157_BREAST C1=CN(C(=O)N=C1N)[C@H]2C([C@@H]([C@H](O2)CO)O)(F)F
MDAMB435S_SKIN CC(C)N(CCCNC(=O)NC1=CC=C(C=C1)C(C)(C)C)C[C@@H]2[C@H]([C@H]([C@@H](O2)N3C=CC4=C3N=CN=C4N)O)O.CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(=O)(=O)C4=CC(=C(C=C4)N[C@H](CCN5CCOCC5)CSC6=CC=CC=C6)S(=O)(=O)C(F)(F)F)C7=CC=C(C=C7)Cl)C
MDAMB157_BREAST CCCC[C@@H](C)/C=C(\C)/C=C(\C)/C(=O)NC1=C[C@]([C@H]2[C@@H](C1=O)O2)(/C=C/C=C/C=C/C(=O)NC3C(=O)CCC3=O)O
MDAMB415_BREAST CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@H]3[C@@H]([C@@](C2(C)C)(C[C@@H]1OC(=O)[C@@H]([C@H](C5=CC=CC=C5)NC(=O)OC(C)(C)C)O)O)OC(=O)C6=CC=CC=C6)(CO4)OC(=O)C)O)C)O
MDAMB231_BREAST CC(C)CNC1=NC(=NC=C1)NCC2=CSC(=N2)C3=CC=CC=C3
MDAMB468_BREAST B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)C2

Using Levenshtein distance, we can replace rcellminer ver cell line to Drug Cell ver cell line!

In [28]:
dist = 100
for i in train[0]:
    if 'MDA' in i:
        distance = Levenshtein.distance(i, 'MDA-MB-231')
        if dist > distance:
            print(i)
            dist = distance

NameError: name 'Levenshtein' is not defined

In [ ]:
dist = 100
for i in train[0]:
    if 'MCF' in i:
        distance = Levenshtein.distance(i, 'MCF7')
        if dist > distance:
            print(i)
            dist = distance

In [ ]:
dist = 100
for i in train[0]:
    distance = Levenshtein.jaro_winkler(i, 'MDA-MB-231')
    if dist > distance:
        print(i)
        dist = distance

In [ ]:
dist = 100
for i in train[0]:
    distance = difflib.SequenceMatcher(None, i, 'MDA-MB-231').ratio()
    if dist > distance:
        print(i)
        dist = distance

In [ ]:
for i, j in zip(train[0], train[1]):
    if 'HOP' in i:
        print(i, j)

In [ ]:
for i, j in zip(train[0], train[1]):
    if 'H226' in i:
        print(i, j)